In [5]:
!pip install keras-utils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for keras-utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2654 sha256=826b860bc58d8f02e3b5938c2048b85c6fd27527aa8a4b4eeab64313fd12cedc
  Stored in directory: c:\users\infog\appdata\local\pip\cache\wheels\04\e4\c5\8eda37877405e9629bdb31de4ae348a0a7386382d7c3eb5975
Successfully built keras-utils



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Path to keypoints directory
keypoints_dir = "video_key_points"

# Load dataset
def load_keypoints(keypoints_dir):
    data, labels = [], []
    label_map = {}  # Mapping gloss to numeric label
    current_label = 0
    
    for split_dir in os.listdir(keypoints_dir):
        split_path = os.path.join(keypoints_dir, split_dir)
        for gloss in os.listdir(split_path):
            gloss_path = os.path.join(split_path, gloss)
            if gloss not in label_map:
                label_map[gloss] = current_label
                current_label += 1

            for npz_file in os.listdir(os.path.join(gloss_path)):
                npz_path = os.path.join(gloss_path, npz_file)
                keypoints = np.load(npz_path)["keypoints"]
                data.append(keypoints)
                labels.append(label_map[gloss])

    return np.array(data, dtype=object), np.array(labels), label_map

data, labels, label_map = load_keypoints(keypoints_dir)

# Pad sequences to ensure equal length
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_SEQ_LENGTH = 50  # Define max sequence length
INPUT_DIM = data[0].shape[1] if len(data) > 0 else 0  # Dimensionality of keypoints

data_padded = pad_sequences(data, maxlen=MAX_SEQ_LENGTH, dtype="float32", padding="post", truncating="post")

# One-hot encode labels
from tensorflow.keras.utils import to_categorical

labels_categorical = to_categorical(labels, num_classes=len(label_map))


In [14]:
MAX_SEQ_LENGTH = 50  # Define max sequence length
INPUT_DIM = data[0].shape[1] if len(data) > 0 else 0  # Dimensionality of keypoints

data_padded = pad_sequences(data, maxlen=MAX_SEQ_LENGTH, dtype="float32", padding="post", truncating="post")

# One-hot encode labels
from tensorflow.keras.utils import to_categorical

labels_categorical = to_categorical(labels, num_classes=len(label_map))

In [15]:
# Train-validation-test split
X_train, X_temp, y_train, y_temp = train_test_split(data_padded, labels_categorical, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
